# 🟡 Introduction: Q-Learning for Pac-Man  

## 📌 Overview  
Reinforcement Learning (RL) is a fundamental paradigm in artificial intelligence where an agent learns to interact with an environment to maximize cumulative rewards. **Q-Learning**, a model-free off-policy algorithm, is one of the most widely used RL techniques for discrete state-action spaces. This notebook presents a Q-learning implementation for training an autonomous agent to play **Pac-Man** using **Pygame**.  

## 🎯 Goal  
The objective of this project is to develop an **intelligent agent** that can navigate the Pac-Man environment optimally by learning from experience, without prior knowledge of the environment’s transition dynamics. The agent will iteratively refine its **Q-table**, improving its decision-making to achieve **higher scores and longer survival times**.  

## 🔬 Why Q-Learning?  
Q-Learning is particularly suitable for this problem due to its ability to:  
✔ Handle **discrete** state-action spaces efficiently.  
✔ Learn from **trial-and-error** interactions without requiring a model of the environment.  
✔ Converge to an **optimal policy** given sufficient exploration and iterations.  

## 🤖 How It Works  
Q-learning operates by updating a Q-table, where each entry **Q(s, a)** represents the expected reward of taking action **a** in state **s**. The core update rule follows:  

$$
Q(s, a) \leftarrow Q(s, a) + \alpha \left( r + \gamma \max_{a'} Q(s', a') - Q(s, a) \right)
$$

Where:  
- $s$, $a$: Current state and action.  
- $s'$, $a'$: Next state and optimal next action.  
- $r$: Reward received after taking action $a$.  
- $\alpha$ (learning rate): Controls how much new information overrides the old.  
- $\gamma$ (discount factor): Weighs the importance of future rewards.  

The agent learns an optimal policy by iteratively refining **Q(s, a)** through experience, balancing **exploration (ε-greedy policy)** and **exploitation (greedy selection)**.  

## 🏗 Structure of This Notebook  
This notebook will be structured as follows:  
1. **Environment Setup** – Defining states, actions, and rewards.  
2. **Q-Learning Implementation** – Training loop, Q-table updates, and exploration strategy.  
3. **Training & Results** – Running simulations, analyzing performance, and fine-tuning hyperparameters.  
4. **Future Enhancements** – Exploring potential improvements such as function approximation (Deep Q-Networks).  


In [ ]:
import numpy as np
from numba import njit
import pygame
import time
import pygame
import sys
import random

# 🏗 Maze Generation for Pac-Man  

## 📌 Overview  
To create a **playable game map**, we generate a **connected maze** using **Prim’s Algorithm** and embed it into a **13×18 grid** with solid outer walls.  

While not the core focus of our **Reinforcement Learning (RL) task**, this step ensures a structured environment where all paths are accessible, making it suitable for training an RL agent.  

---

## 🎲 Function: `generate_connected_maze()`  

### 🔍 Purpose  
Generates a **9×14 maze** using **Prim’s Algorithm**, ensuring all **dots ('D')** are connected.  

### 🛠 Process  
1. **Initialize Grid** → Fill with **walls ('W')**.  
2. **Start Carving Paths** → Use Prim’s Algorithm to progressively replace walls with **dots ('D')**.  
3. **Ensure Connectivity** → No isolated regions exist; Pac-Man can reach all areas.  

### 📤 Output  
Returns a **NumPy array (9×14) with 'W' (walls) and 'D' (dots)**.  


In [ ]:
def generate_connected_maze(inner_rows = 9, inner_cols = 14, start_row = 4, start_col = 7):

    """
    Generates a connected maze using Prim's algorithm in the inner area (9x14).
    Walls ('W') and dots ('D') are placed such that all dots are reachable from Pac-Man's starting position.

    Args:
        inner_rows (int): Rows of the playable area (default: 9).
        inner_cols (int): Columns of the playable area (default: 14).
        start_row (int): Pac-Man's starting row in the inner grid (default: 4).
        start_col (int): Pac-Man's starting column in the inner grid (default: 7).

    Returns:
        np.ndarray: Inner grid (14x9) with 'W' (walls) and 'D' (dots).
    """

    # Initialize inner grid with walls ('W')
    inner_grid = np.full((inner_rows, inner_cols), 'W', dtype = '<U1')

    # Directions: Up, Down, Left, Right (relative movement)
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    # Track walls that can be carved to connect paths
    walls = []

    # Mark Pac-Man's starting position as a path ('D')
    inner_grid[start_row, start_col] = 'D'

    # Add neighboring walls of the starting cell to the wall list
    for dx, dy in directions:
        neighbor_row = start_row + dx
        neighbor_col = start_col + dy
        if 0 <= neighbor_row < inner_rows and 0 <= neighbor_col < inner_cols:
            walls.append((neighbor_row, neighbor_col, start_row, start_col))

    # Process walls until none remain
    while walls:
        # Randomly select a wall to potentially carve
        wall_idx = random.randrange(len(walls))
        wall_row, wall_col, parent_row, parent_col = walls.pop(wall_idx)


        # Calculate the cell "opposite" to the parent through the wall, this is how we ensure connectivity (carving a path through the wall)
        opp_row = wall_row + (wall_row - parent_row)
        opp_col = wall_col + (wall_col - parent_col)

        # Check if the opposite cell is within bounds
        if 0 <= opp_row < inner_rows and 0 <= opp_col < inner_cols:

            # If the opposite cell is a wall, carve a path, carve the current wall and opposite cell into paths
            if inner_grid[opp_row, opp_col] == 'W':
                inner_grid[wall_row, wall_col] = 'D'
                inner_grid[opp_row, opp_col] = 'D'

                # Add neighboring walls of the opposite cell to the wall list
                for dx, dy in directions:
                    new_row = opp_row + dx
                    new_col = opp_col + dy
                    if 0 <= new_row < inner_rows and 0 <= new_col < inner_cols:
                        walls.append((new_row, new_col, opp_row, opp_col))

    return inner_grid

## 🗺 Function: `generate_map()`  

### 🔍 Purpose  
Creates the **full game map (13×18)** by embedding the generated maze into a **walled structure**.  

### 🛠 Steps  
1. **Initialize a Blank Grid (13×18)** → Filled with empty spaces.  
2. **Set Outer Borders** → Walls ('W') define the boundary.  
3. **Generate Inner Maze (9×14)** → Calls `generate_connected_maze()`.  
4. **Modify Walls for Loops** → Converts ~35% of walls into dots to introduce loops.  
5. **Embed the Maze into the Full Grid** → Placed at `[2:11, 2:16]`.  

### 📤 Output  
Returns a **13×18 NumPy array** representing the **final game map**.  

In [ ]:
def generate_map():
    """
    Builds the full 13x18 map with:
    - 2 Layer Edges as walls ('W').
    - Inner area as a connected maze with dots ('D') and Pac-Man ('P').
    """
    # Initialize full grid with empty spaces
    full_grid = np.full((13, 18), ' ', dtype='<U1')

    # Set outer edges to walls
    full_grid[:2, :] = 'W'
    full_grid[:, :2] = 'W'   # Left column
    full_grid[-2:, :] = 'W'  # Bottom row
    full_grid[:, -2:] = 'W'  # Right column

    # Generate the inner maze (14x9 playable area)
    inner_maze = generate_connected_maze()

    for i in range(inner_maze.shape[0]):
        for j in range(inner_maze.shape[1]):
            if inner_maze[i][j] == 'W':
                rnd = np.random.rand()
                if rnd < 0.45:
                    inner_maze[i][j] = 'D'

    # Place the inner maze into the full grid (rows 1-14, columns 1-9)
    full_grid[2:11, 2:16] = inner_maze

    return full_grid

# 🔢 `Conv3` Function: Encoding Pac-Man's State into a Numeric Representation  

## 📌 Overview  
In our **Q-Learning implementation**, we need to efficiently represent the **state** of the environment as a **single numeric value** (index) in the **Q-table**.  

The function `Conv3()` **encodes** a given **state representation** (a combination of 'D', 'W', 'E') into a **unique number** using **base-3 conversion**.  

---

## 🎯 Function: `Conv3(combination)`

### 🔍 Purpose  
Maps a **state representation** (12 locations around Pac-Man) into a **single number**, which serves as the **index** for our **Q-table**.  

### 📥 Input  
A list of **12 elements**, where each element is one of:  
- **'E'** → **Empty space**  
- **'D'** → **Dot**  
- **'W'** → **Wall**  

These elements define **all locations** Pac-Man can **access within a distance of 2**:

\begin{align*}
    \begin{pmatrix}
        - & - & C_1 & - & - \\
        - & C_2 & C_3 & C_4 & - \\
        C_5 & C_6 & \text{Agent} & C_7 & C_8 \\
        - & C_9 & C_{10} & C_{11} & - \\
        - & - & C_{12} & - & - \\
    \end{pmatrix}
\end{align*}

So, if we consider the agent at the center (as shown in the above), the $C_1$ through $C_{12}$ define the state of the agent.


### 📤 Output  
A **unique integer** representing the **state**, computed using a **base-3 encoding**.  

---

## 🛠 How It Works  
### 🔢 Encoding Process  
1. **Convert Symbols to Numeric Values**  
   - 'E' → **0**  
   - 'D' → **1**  
   - 'W' → **2**  
   
2. **Apply Base-3 Conversion**  
   - The **rightmost element** represents **\(3^0\)**,  
   - The **next** represents **\(3^1\)**,  
   - And so on, forming a **base-3 number**.  

   **Example Calculation:**  
   ```
   Combination: ['D', 'E', 'W', 'D']
   Encoded:     [1, 0, 2, 1]
   Decimal Value = (1 × 3³) + (0 × 3²) + (2 × 3¹) + (1 × 3⁰)
                = (27) + (0) + (6) + (1)
                = 34
   ```

---

## 📌 Why Use Base-3 Encoding?  
- **Compact Representation**: Reduces a **12-character state** to a **single number**.  
- **Fast Lookup**: Q-table indexing becomes efficient.  
- **Unique Mapping**: Each combination produces a **distinct** index.  

In [ ]:
def Conv3( combination ):

    """
    Conv3() function works as following:
        Input:
            - Combination: Is an array of element, and each element is one of the following:
                1. E: empty
                2. D: dot
                3. W: wall
        OutPut:
            - The out put of this function is a number which is the hase of the initial combination
    """

    coded_comb = [0 for i in range(len(combination))]

    encoder = {'E': 0, 'D': 1, 'W': 2}

    for index in range(len(combination)):
        coded_comb[index] = encoder[combination[index]]

    decimal, power = 0, 0
    index = len(coded_comb) - 1

    while( index >= 0 ):
        decimal += coded_comb[index] * 3**power
        index -= 1
        power += 1

    return(decimal)

## 🔢 Q-Table Definition

The **Q-Table** is a **3D matrix** with the shape:

$$
(3^{12}, 4, 4)
$$

### 📌 Explanation of Dimensions:
1. **$3^{12}$ (Number of States)**  
   - Each state is defined by **12 neighboring positions** around Pac-Man.  
   - Each position can have **3 possible values**:
     - `E` (Empty)  
     - `D` (Dot)  
     - `W` (Wall)  
   - Since there are **12 positions**, the total number of states is:

     $$
     3^{12} = 531441
     $$

2. **First "4" (Previous Action)**  
   - Represents the **previous move** Pac-Man took:
     - `0`: Up  
     - `1`: Down  
     - `2`: Left  
     - `3`: Right  

3. **Second "4" (Current Action)**  
   - Represents the **current action** Pac-Man is evaluating:
     - `0`: Up  
     - `1`: Down  
     - `2`: Left  
     - `3`: Right  

### 🚀 Purpose:
- This structure **prevents Pac-Man from getting stuck in loops** by tracking both the **previous action** and **current action**.
- It helps **reinforce efficient movement strategies** and **avoid unnecessary back-and-forth motions**.

In [ ]:
universal_mat = np.zeros((3**12, 4, 4))

## 🎯 Reward Function

The **reward function** guides the **Pac-Man agent's behavior** by assigning rewards based on its movement and interactions with the environment.

### 📌 Reward Rules:
- **Moving into an empty space (`E`)** → **Small negative reward** (`-0.5`)  
- **Reversing direction (opposite of `prev_action`)** → **Larger negative reward** (`-1`)  
- **Eating a dot (`D`)** → **Positive reward** (`4`)  
- **Hitting a wall (`W`)** → **Not allowed** (the action is removed)

---



In [ ]:
def reward_calc(state, action, prev_action):
    """
    reward_calc() function:
        - Determines the reward for a given action in a given state.

    Parameters:
        - state: A list of size 12 representing the agent's neighborhood.
                 Example: ['W', 'W', 'D', 'D', 'E', 'D', 'W', 'E', 'D', 'E', 'D']
        - action: The action the agent is about to take ('U', 'D', 'L', 'R').
        - prev_action: The agent's previous action (used to prevent backtracking).

    Returns:
        - Reward value (float).
    """

    # Small penalty for moving into an empty space
    empty_penalty = -0.5
    if action == 'U' and state[2] == 'E':
        return empty_penalty
    elif action == 'D' and state[9] == 'E':
        return empty_penalty
    elif action == 'L' and state[5] == 'E':
        return empty_penalty
    elif action == 'R' and state[6] == 'E':
        return empty_penalty

    # Larger penalty for reversing direction
    reverse_penalty = -1
    if (action == 'R' and prev_action == 'L') or \
       (action == 'L' and prev_action == 'R') or \
       (action == 'U' and prev_action == 'D') or \
       (action == 'D' and prev_action == 'U'):
        return reverse_penalty

    # Positive reward for eating a dot
    return 4

# 🔄 `update_state()` Function Documentation

## Overview
The `update_state()` function updates the agent’s position and state after executing an action in the game environment. It ensures that the agent moves correctly without colliding with walls (`W`), updates the game map accordingly, and extracts a new state representation based on the agent's surroundings.

---

## Parameters

- **agent_loc** (`list[int]`):  
  The agent's current location in the game grid.  
  *Example*: `[1, 7]` (indicating row 1, column 7).

- **action** (`str`):  
  The action the agent intends to take.  
  *Possible values*:  
  - `'U'` for Up  
  - `'D'` for Down  
  - `'L'` for Left  
  - `'R'` for Right

- **game_map** (`2d numpy.array`):  
  The current game map represented as a 2D list of strings, where:  
  - `'E'` represents an empty space,  
  - `'D'` represents a dot,  
  - `'W'` represents a wall,  
  - `'A'` represents the agent.

---

## Workflow

1. **Validate Movement**  
   - The function checks if moving in the specified direction is allowed by ensuring that the adjacent cell in that direction is not a wall (`W`).  
   - If the cell is not a wall, the agent's location is updated accordingly.

2. **Update Agent Location**  
   - A copy of the current agent location is made, and based on the action, the agent's new location (`agent_updated_loc`) is computed.

3. **Modify the Game Map**  
   - The agent's previous location is set to empty (`'E'`).  
   - The new location is updated to mark the agent's presence with `'A'`.

4. **Extract the Updated State**  
   - A 3×3 grid centered around the agent's new position is extracted from the updated game map.  
   - Four additional positions are inserted to capture:
     - The cell 2 steps above the agent,
     - The cell 2 steps to the left,
     - The cell 2 steps to the right,
     - The cell 2 steps below.
   - The agent’s own cell is then removed from this list, and the final state is converted into a NumPy array (`agent_updated_state`).

---

## Returns

- **agent_updated_loc** (`list[int]`):  
  The new location of the agent after performing the action.

- **agent_updated_state** (`numpy.array`):  
  The updated state representation, capturing the agent's immediate surroundings (a combination of the 3×3 grid and additional 4 positions).

- **game_map** (`2d numpy.array`):  
  The updated game map after the agent's move.


In [ ]:
def update_state(agent_loc, action, game_map):

    """
    The update_state() function works as following:
        Input:
            1. agent_loc: represent the location of the agent in the game map
                example:
                    [1, 7]: means the agent is in the first row and seventh column

            2. action: represents the agent action (i.e. Up, Down, Left, Right)
            3. game_map: represents the map of the game

        Output:
            1. agent_updated_loc: the new location of the agent after doing an action.
            2. agent_updated_state: the new state of the agent.
            3. game_map: the updated version of the game map
    """


    agent_updated_loc = agent_loc.copy()

    if( action == 'U' and game_map[agent_loc[0] - 1][agent_loc[1]] != 'W' ):
        agent_updated_loc[0] -= 1

    elif( action == 'D' and game_map[agent_loc[0] + 1][agent_loc[1]] != 'W' ):
        agent_updated_loc[0] += 1

    elif( action == 'L' and game_map[agent_loc[0]][agent_loc[1] - 1] != 'W' ):
        agent_updated_loc[1] -= 1

    elif( action == 'R' and game_map[agent_loc[0]][agent_loc[1] + 1] != 'W' ):
        agent_updated_loc[1] += 1

    game_map[agent_loc[0]][agent_loc[1]] = 'E'
    game_map[agent_updated_loc[0]][agent_updated_loc[1]] = 'A'

    agent_updated_state = []
    for row in range(3):
        for col in range(3):
            agent_updated_state.append(game_map[agent_updated_loc[0] - 1 + row][agent_updated_loc[1] - 1 + col])

    agent_updated_state.insert(0, game_map[agent_updated_loc[0] - 2][agent_updated_loc[1]])
    agent_updated_state.insert(4, game_map[agent_updated_loc[0]][agent_updated_loc[1] - 2])
    agent_updated_state.insert(8, game_map[agent_updated_loc[0]][agent_updated_loc[1] + 2])
    agent_updated_state.insert(12, game_map[agent_updated_loc[0] + 2][agent_updated_loc[1]])

    del agent_updated_state[6]
    agent_updated_state = np.array(agent_updated_state)

    return(agent_updated_loc, agent_updated_state, game_map)

# choose_act() Function Documentation

## Overview
The `choose_act()` function selects an action for the agent based on the current state's Q-values and an exploration strategy. It first removes invalid actions (e.g., those blocked by walls) from consideration. Then, using a dynamic threshold (similar to an epsilon-greedy approach), it either chooses the action with the highest Q-value among the available ones or randomly selects an action. Finally, it encodes the chosen action into a corresponding direction ('U', 'D', 'L', or 'R').

---

## Parameters

- **possible_acts** (`array`):  
  An array representing a row of the Q-table. It contains Q-values for each of the four actions (indexed as 0, 1, 2, 3 for 'U', 'D', 'L', and 'R', respectively) for the current state.

- **tot_ep** (`int`):  
  The total number of episodes planned for training. This value is used to adjust the exploration threshold dynamically.

- **curr_ep** (`int`):  
  The current episode number. Used with `tot_ep` to calculate a decaying threshold for exploration.

- **threshold_init** (`float`):  
  The initial threshold (epsilon) value that determines the probability of selecting a random action. This value decays over time to favor exploitation over exploration.

- **state** (`array`):  
  The state representation of the agent, an array (of size 12) that includes information about the agent's surrounding cells. Specific indices in the array correspond to cells in particular directions:
  - **Index 2**: Cell above (for action 'U').
  - **Index 5**: Cell to the left (for action 'L').
  - **Index 6**: Cell to the right (for action 'R').
  - **Index 9**: Cell below (for action 'D').

---

## Workflow

1. **Initialize Available Actions:**  
   Start with all four possible actions


2. **Remove Invalid Actions:**  
- If `state[2]` is `'W'`, remove action `0` (Up).
- If `state[5]` is `'W'`, remove action `2` (Left).
- If `state[6]` is `'W'`, remove action `3` (Right).
- If `state[9]` is `'W'`, remove action `1` (Down).

3. **Compute Dynamic Threshold:**  
Calculate the threshold to balance exploration and exploitation.

This value decreases as the training progresses, reducing the probability of taking a random action.

4. **Select Action:**  
- Generate a random number between 0 and 1.
- **If the random number is greater than the threshold:**  
  - Sort the indices of `possible_acts` in descending order (i.e., highest Q-value first).
  - Iterate over these sorted indices and choose the first action that is in the list of available actions.
- **Else:**  
  - Choose a random action from the available actions.

5. **Encode Action:**  
Convert the numerical action (0, 1, 2, or 3) into a direction using:


In [ ]:
def choose_act(possible_acts, tot_ep, curr_ep, threshold_init, state):
    """
    choose_act() function works as following:
        Input:
            It takes a single input which is an array corresponds to a row of the Q table.

        OutPut:
            The output of this function is an action that we are going to do (i.e. Up, Down, Left, Right)

    This function returns the action with maximum Q by 0.9 probability, and with 0.1 probability, returns a random action.
    """

    availables = np.array([0, 1, 2, 3])
    if(state[2] == 'W'):
        availables = np.delete(availables, np.argwhere(np.isin(availables, 0)))
    if(state[5] == 'W'):
        availables = np.delete(availables, np.argwhere(np.isin(availables, 2)))
    if(state[6] == 'W'):
        availables = np.delete(availables, np.argwhere(np.isin(availables, 3)))
    if(state[9] == 'W'):
        availables = np.delete(availables, np.argwhere(np.isin(availables, 1)))

    threshold = threshold_init * (tot_ep - curr_ep) / tot_ep
    rand_num = np.random.rand()

    if(rand_num > threshold):
        sorted_acts_idx = np.argsort(possible_acts)[::-1]
        for i in sorted_acts_idx:
            if(np.isin(availables, i).any()):
                action = i
                break
    else:
        action = np.random.randint(1, 5) - 1

    action_encoder = {0: 'U', 1: 'D', 2: 'L', 3: 'R'}
    return(action_encoder[action])

# init_agent() Function Documentation

## Overview
The `init_agent()` function initializes the agent at a random position within the given map (`mapp`). The function ensures that the agent does not spawn inside a wall ('W') by continuously generating random coordinates until a valid position is found. This random initialization helps increase exploration by allowing the agent to start in different locations across multiple runs.

---

## Parameters

- **mapp** (`2D array`):  
  A matrix representing the environment, where each cell corresponds to a position in the map.  
  - `'W'` represents a wall (invalid position for the agent).
  - Other values represent free spaces where the agent can be placed.

---

## Workflow

1. **Randomly Generate Coordinates:**  
   - The function selects random `x` and `y` coordinates within predefined bounds:
     ```python
     x = np.random.randint(2, 11)
     y = np.random.randint(2, 15)
     ```
     These bounds ensure the agent is placed within the main playable area of the map.

2. **Check for Validity:**  
   - The function verifies that the randomly chosen position is not occupied by a wall (`'W'`).  
   - If the position is invalid, it repeats the process until a valid position is found.

3. **Place the Agent:**  
   - Once a valid position is determined, the agent ('A') is placed at `(x, y)`:
     ```python
     mapp[x, y] = 'A'
     ```

4. **Return Values:**  
   - The function returns:
     - The agent's starting position as a list `[x, y]`.
     - The updated map with the agent placed.

---

## Returns

- **position** (`list`):  
  The randomly chosen valid starting position of the agent as `[x, y]`.

- **mapp** (`2D array`):  
  The updated map with the agent ('A') placed at the chosen position.


In [ ]:
def init_agent(mapp):

    while True:
        x = np.random.randint(2, 11)
        y = np.random.randint(2, 15)

        if( mapp[x, y] != 'W' ):
            break
    mapp[x, y] = 'A'

    return [x, y], mapp

# Reinforcement Learning Training Loop Documentation

## Overview  
This training loop is designed to train an agent using Q-learning in a grid-based environment with dots and walls. The agent learns to navigate the environment while maximizing rewards.

---

## Training Process  

### **1. Map Initialization**  
- The environment consists of different maps (`map_id` varies from 0 to 2).  
- The map is generated using `generate_map()`, and a copy (`map_reset`) is stored for resetting after each episode.  
- The agent starts with an initial action (`prev_action = 'U'`) and a state index (`prev_stated_index = 0`).  

---

### **2. Episode Execution**  
Each episode runs for **1500 iterations** (`programm_counter` varies from 0 to 1499). The following steps occur:

#### **2.1. Reset Game State**  
- The `game_map` is reset.  
- The number of dots (`dots_number`) in the environment is counted.  
- Hyperparameters `alpha` (learning rate) and `gamma` (discount factor) are set.  
- The agent is placed at a random valid location using `init_agent()`.  
- An `action_counter` tracks the number of moves per episode.

#### **2.2. Running the Game Loop**  
The game runs until one of the following conditions is met:  
1. No dots remain (`dots_number == 0`).  
2. The agent exceeds **2000** moves (`action_counter >= 2000`).  

In each iteration:  
1. **Extract Agent State**  
   - The local **3×3** grid around the agent is captured.  
   - Four additional positions (up, left, right, down) are added.  
   - The state is converted to an index using `Conv3(state)`.  

2. **Choose an Action**  
   - The action is chosen using `choose_act()`, which balances exploration and exploitation using an epsilon-decay strategy.  
   - Exploration probability decreases as training progresses.  

3. **Calculate Reward**  
   - The reward is determined by `reward_calc(state, action, prev_action)`.  
   - If the agent collects a dot, `dots_number` is decremented.  

4. **Update the Game State**  
   - The agent's new location, state, and updated `game_map` are obtained using `update_state(agent_location, action, game_map)`.  
   - The new state is converted to an index using `Conv3(updated_state)`.  

5. **Update the Q-Table**  
   - The Q-value for the chosen action is updated using the Q-learning formula:  
    \begin{align*}
        Q(s, a) = (1-\alpha) Q(s, a) + \alpha(R + \gamma \max Q(s', a'))
    \end{align*}
   - The update ensures that the agent gradually improves its action-selection policy.  

6. **Update Agent Location & Previous Action**  
   - The agent moves to `updated_location`.  
   - The `action_counter` increments.  
   - `prev_action` is updated for the next iteration.  

---

In [ ]:
for map_id in range(2):
    initial_map = generate_map()
    map_reset = initial_map
    print("MAP:", map_id)

    prev_action = 'U'                                        # <----- SetUp Previous Action
    prev_stated_index = 0

    for programm_counter in range(1500):

        game_map = map_reset.copy()
        dots_number = np.sum(game_map == 'D')
        alpha = 0.3                                          # <----- Define the hyperparameter alpha and gamma.
        gamma = 0.5

        agent_location, game_map = init_agent(game_map)      # <----- The agent location is the initial position of the agent
        action_counter = 0                                   # <----- Count the number of moves in a single game


        # We perform the game unitl one of the following condtions met:
            # 1. No dots remains
            # 2. The game run longer than 1000 moves
        while( dots_number > 0 and action_counter < 2000 ):

            state = []
            for row in range(3):
                for col in range(3):
                    state.append(game_map[agent_location[0] - 1 + row][agent_location[1] - 1 + col])

            state.insert(0, game_map[agent_location[0] - 2][agent_location[1]])
            state.insert(4, game_map[agent_location[0]][agent_location[1] - 2])
            state.insert(8, game_map[agent_location[0]][agent_location[1] + 2])
            state.insert(12, game_map[agent_location[0] + 2][agent_location[1]])

            del state[6]
            state = np.array(state)
            state_index = Conv3( state )

            # Choose an action:
            action = choose_act(universal_mat[state_index][prev_stated_index], 1500, programm_counter, max(1-(map_id+6)/10, 0.05), state)

            # Calculate the reward:
            reward = reward_calc(state, action, prev_action)

            if( reward == 4 ):
                dots_number -= 1

            # Update the Q-table & game map:   update_state(agent_loc, action, game_map)
            updated_location, updated_state, game_map = update_state(agent_location, action, game_map)
            updated_state_index = Conv3(updated_state)

            # Update the Q-table:
            action_map = {'U': 0, 'D': 1, 'L': 2, 'R': 3}
            action_index = action_map[action]
            if(prev_action != 'None'):
                prev_action_index = action_map[prev_action]
                universal_mat[state_index][prev_action_index][action_index] = ((1-alpha)*universal_mat[state_index][prev_action_index][action_index]) + (alpha*(reward+gamma*np.max(universal_mat[updated_state_index][prev_action_index])))

            # Update the location:
            agent_location = updated_location

            # We made a move so incerase the action counter:
            action_counter += 1
            prev_action = action

# Visualization Functions

This section documents the functions used for visualizing the Pac-Man game: `draw_pacman`, `draw_map`, and `grid_to_pixel`.

---

## `draw_pacman(screen, center, radius, mouth_angle)`

**Purpose:**  
Draws the Pac-Man character with an animated mouth on the given Pygame screen. The mouth is animated by drawing a wedge (polygon) that simulates an opening and closing effect.

**Parameters:**  
- **`screen`**: The Pygame surface where Pac-Man will be drawn.
- **`center`**: A tuple `(x, y)` representing the center coordinates of Pac-Man.
- **`radius`**: An integer for the radius of Pac-Man.
- **`mouth_angle`**: A float (in radians) representing the total angle of the mouth opening. This angle is typically oscillated over time to create the animation.

**Notes:**  
- Pac-Man is drawn as a filled circle in yellow.
- A wedge (polygon) is drawn in the background color (BLACK) over the circle to create the mouth effect.

In [ ]:
# --- Constants ---
WIDTH, HEIGHT = 40, 40
WHITE = (255, 255, 255)
WALL_COLOR = (50, 50, 50)        # Dark grey for walls
YELLOW = (255, 255, 0)           # Bright yellow for Pac-Man
BLACK = (0, 0, 0)              # Black background
PELLET_COLOR = (200, 200, 200)  # Light grey for pellets

# Mouth animation parameters
MAX_MOUTH_ANGLE = 0.6  # Maximum opening angle in radians
MOUTH_FREQUENCY = 1    # Frequency of oscillation in Hz

# Movement animation parameters
ANIM_FRAMES = 10       # Number of frames for moving between cells

# Initialize Pygame
pygame.init()
font = pygame.font.Font(None, 36)



# --- Pac-Man Drawing with Animated Mouth ---
def draw_pacman(screen, center, radius, mouth_angle):
    """
    Draws Pac-Man with an animated mouth.
    
    Args:
        screen: Pygame surface to draw on.
        center: Tuple (x, y) representing the center of Pac-Man.
        radius: Radius of Pac-Man.
        mouth_angle: Total angle (in radians) of the mouth opening.
    """
    # Draw the full Pac-Man circle
    pygame.draw.circle(screen, YELLOW, center, radius)
    
    half_angle = mouth_angle / 2.0
    # Pac-Man faces right (0 radians)
    start_angle = half_angle
    end_angle = -half_angle

    x, y = center
    point1 = (x + radius * math.cos(start_angle), y - radius * math.sin(start_angle))
    point2 = (x + radius * math.cos(end_angle), y - radius * math.sin(end_angle))
    
    # Draw a polygon (mouth) in the background color
    pygame.draw.polygon(screen, BLACK, [center, point1, point2])

## `draw_map(screen, map_data, current_mouth_angle, animated_position=None)`

**Purpose:**  
Visualizes the game map on the Pygame screen. It draws the environment by iterating through the grid and rendering walls, pellets, and Pac-Man. If an `animated_position` is provided, it draws Pac-Man at that pixel location to allow smooth movement transitions.

**Parameters:**  
- **`screen`**: The Pygame screen object where the game map is rendered.
- **`map_data`**: A 2D NumPy array (or similar structure) representing the full game map.
  - The map includes cells like `'W'` (wall), `'D'` (dot/pellet), and `'A'` (agent/Pac-Man).
- **`current_mouth_angle`**: The current mouth opening angle (in radians) for animating Pac-Man.
- **`animated_position`** (optional): A tuple `(x, y)` representing the pixel coordinates where Pac-Man should be drawn. If not provided, Pac-Man is drawn at the default grid location.

**Notes:**  
- Only the interior of the map (excluding the first and last rows/columns) is rendered for a cleaner display.
- Walls are drawn as dark grey rectangles, pellets as small light grey circles, and Pac-Man with the animated drawing provided by `draw_pacman`.

---

## `grid_to_pixel(cell_row, cell_col)`

**Purpose:**  
Converts grid cell coordinates from the game map to pixel coordinates on the Pygame screen for the interior display. This is useful for aligning the drawing of moving objects (like Pac-Man) with the grid.

**Parameters:**  
- **`cell_row`**: The row index of the cell in the game map.
- **`cell_col`**: The column index of the cell in the game map.

**Returns:**  
- A tuple `(pixel_x, pixel_y)` representing the pixel coordinates of the center of the specified grid cell.

**Notes:**  
- The function adjusts for the fact that the displayed area excludes the outer edges of the map.
- The conversion ensures that objects drawn on the grid align perfectly with the rendered map cells.


In [ ]:
# --- Map Drawing Function (with optional animated agent) ---
def draw_map(screen, map_data, current_mouth_angle, animated_position=None):
    """
    Visualizes the Pac-Man game using Pygame.
    Displays only the interior of the map (excluding first and last rows/columns).
    If animated_position is provided, the agent ('A') is drawn at that position instead.
    
    Args:
        screen: Pygame screen object.
        map_data: The full game map matrix.
        current_mouth_angle: Current mouth opening angle for Pac-Man.
        animated_position (tuple): (x, y) pixel coordinates to draw Pac-Man.
    """
    screen.fill(BLACK)
    rows, cols = map_data.shape
    for y in range(1, rows - 1):
        for x in range(1, cols - 1):
            cell = map_data[y, x]
            # Adjust position: interior drawing offset
            rect = pygame.Rect((x - 1) * WIDTH, (y - 1) * HEIGHT, WIDTH, HEIGHT)
            
            if cell == 'W':
                pygame.draw.rect(screen, WALL_COLOR, rect)
            elif cell == 'D':
                pygame.draw.circle(screen, PELLET_COLOR, rect.center, 5)
            elif cell == 'A':
                # Skip drawing the agent if an animated position is provided.
                if animated_position is None:
                    pygame.draw.circle(screen, YELLOW, rect.center, WIDTH // 3)
                    
    # Draw animated Pac-Man if position is provided.
    if animated_position is not None:
        draw_pacman(screen, animated_position, WIDTH // 3, current_mouth_angle)



# --- Helper: Compute Pixel Center for a given grid cell ---
def grid_to_pixel(cell_row, cell_col):
    """
    Converts a grid cell position (row, col) from the full map to pixel coordinates
    for the interior display. Interior starts at row=1, col=1 in map_data.
    """
    # Adjusted pixel center for interior display:
    pixel_x = (cell_col - 1) * WIDTH + WIDTH // 2
    pixel_y = (cell_row - 1) * HEIGHT + HEIGHT // 2
    return (pixel_x, pixel_y)

# `run_game()`
The final function is the `run_game()` function where we put all things together and simulate the PacMan

In [ ]:
# --- Main Game Loop with Smooth Transition Animation ---
def run_game():
    """
    Runs the main game loop with enhanced graphics, animated Pac-Man,
    and smooth movement transitions.
    """
    # Generate the initial game map and initialize dots count and agent position
    game_map = generate_map()  # Assume defined elsewhere
    dots_number = np.sum(game_map == 'D')
    agent_location = [6, 9]
    prev_action = 'U'
    action_dictionary = {'U': 0, 'D': 1, 'L': 2, 'R': 3}
    
    counter = 0
    total_reward = 0
    agent_last10_ind_history = np.zeros(10, dtype=int)
    
    # Calculate display size for interior (exclude edges)
    interior_rows = game_map.shape[0] - 2
    interior_cols = game_map.shape[1] - 2
    window_size = (WIDTH * interior_cols, HEIGHT * interior_rows)
    screen = pygame.display.set_mode(window_size)
    pygame.display.set_caption("Pac-Man AI")
    
    clock = pygame.time.Clock()
    start_time = time.time()  # For mouth animation timing
    
    running = True
    while dots_number > 0 and running:
        clock.tick(3)  # 30 FPS for smooth animation
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        
        # Calculate current mouth angle using sine oscillation
        elapsed = time.time() - start_time
        current_mouth_angle = MAX_MOUTH_ANGLE * (0.5 + 0.5 * math.sin(2 * math.pi * MOUTH_FREQUENCY * elapsed))
        
        # Build state representation (same as before)
        state = []
        for row in range(3):
            for col in range(3):
                state.append(game_map[agent_location[0] - 1 + row][agent_location[1] - 1 + col])
        state.insert(0, game_map[agent_location[0] - 2][agent_location[1]])
        state.insert(4, game_map[agent_location[0]][agent_location[1] - 2])
        state.insert(8, game_map[agent_location[0]][agent_location[1] + 2])
        state.insert(12, game_map[agent_location[0] + 2][agent_location[1]])
        del state[6]
        state = np.array(state)
        state_index = Conv3(state)  # Assume defined elsewhere
        agent_last10_ind_history[counter % 10] = state_index
        
        # Choose an action using the Q-table
        action = choose_act(universal_mat[state_index][action_dictionary[prev_action]], 1, 0, 0.2, state)
        
        # Calculate reward
        reward = reward_calc(state, action, prev_action)
        prev_action = action
        total_reward += reward
        if reward == 4:
            dots_number -= 1
        
        # Store the current agent location before updating
        old_location = agent_location.copy()
        # Update game state and agent location
        updated_location, updated_state, game_map = update_state(agent_location, action, game_map)
        agent_location = updated_location
        counter += 1
        
        # Animate smooth transition from old_location to new agent_location
        old_pixel = grid_to_pixel(old_location[0], old_location[1])
        new_pixel = grid_to_pixel(agent_location[0], agent_location[1])
        
        for frame in range(ANIM_FRAMES):
            t = frame / float(ANIM_FRAMES)  # interpolation factor [0,1]
            interp_x = int(old_pixel[0] + (new_pixel[0] - old_pixel[0]) * t)
            interp_y = int(old_pixel[1] + (new_pixel[1] - old_pixel[1]) * t)
            animated_position = (interp_x, interp_y)
            
            # Draw map; pass animated_position so Pac-Man is drawn there
            draw_map(screen, game_map, current_mouth_angle, animated_position=animated_position)
            score_text = font.render(f"Score: {total_reward}", True, WHITE)
            screen.blit(score_text, (15, 15))
            pygame.display.flip()
            clock.tick(30)
        
        # After animation, update display one final time with static position
        draw_map(screen, game_map, current_mouth_angle)
        score_text = font.render(f"Score: {total_reward}", True, WHITE)
        screen.blit(score_text, (15, 15))
        pygame.display.flip()
    
    pygame.quit()

# Run the game
run_game()